# Checking the Hartree-Fock Stability conditions

First, we import GQCPy.

In [1]:
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

## The RHF stability conditions

We will run an RHF calculation on $H_4$.

In [2]:
def RHF_calculation(molecule, basis_set='sto-3g'):
    N = molecule.numberOfElectrons()
    basis = gqcpy.RSpinOrbitalBasis_d(molecule, basis_set)
    S = basis.quantize(gqcpy.OverlapOperator())
    
    rsq_hamiltonian = gqcpy.RSQHamiltonian_d.Molecular(basis, molecule)
    objective = gqcpy.DiagonalRHFFockMatrixObjective_d(rsq_hamiltonian, 1.0e-5)  
    
    environment = gqcpy.RHFSCFEnvironment_d.WithCoreGuess(N, rsq_hamiltonian, S) 
    solver = gqcpy.RHFSCFSolver_d.Plain(threshold=1.0e-06, maximum_number_of_iterations=1000)

    qc_structure = gqcpy.RHF_d.optimize(objective, solver, environment)
    nuc_rep = gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()
    print(qc_structure.groundStateEnergy() + nuc_rep)
    
    return qc_structure

In [3]:
H4 = gqcpy.Molecule.HRingFromDistance(4, 1.889, 0)

In [4]:
RHF_model = RHF_calculation(H4).parameters()

-1.694774380710066


These parameters contain everything there is to know about the RHF wave function model. We can now ask this model to calculate the stability matrices. For that we need a Hamiltonian in the RHF MO basis.

In [5]:
restricted_basis = gqcpy.RSpinOrbitalBasis_d(H4, 'sto-3g')
restricted_hamiltonian = gqcpy.RSQHamiltonian_d.Molecular(restricted_basis, H4)
rsq_hamiltonian_mo = restricted_hamiltonian.transformed(RHF_model.expansion())
restricted_stability_matrices = RHF_model.calculateStabilityMatrices(rsq_hamiltonian_mo)

Now we can check the stabilities. All the stability checks are done by the stability matrices themselves.

In [6]:
restricted_stability_matrices.isInternallyStable()

False

In [7]:
restricted_stability_matrices.isExternallyStable()

False

Since the wave function model is not externally stable, we can verify which external instability it contains.

In [8]:
restricted_stability_matrices.isTripletStable()

False

In [9]:
restricted_stability_matrices.isComplexConjugateStable()

False

We can also print the stability description. Note that this runs the calculation of diagonalizing the stability matrix.

In [10]:
restricted_stability_matrices.printStabilityDescription()

The real valued RHF wavefunction contains an internal instability.
The real valued RHF wavefunction contains a real->complex instability.
The real valued RHF wavefunction contains a restricted->unrestricted instability.


This indicates that there are two ways to further lower the energy. Move towards an unrestricted formalism, or use complex valued parametrs. We will opt for the second.

In [25]:
def complex_RHF_calculation(molecule, basis_set='sto-3g'):
    N = molecule.numberOfElectrons()
    basis = gqcpy.RSpinOrbitalBasis_cd(molecule, basis_set)
    S = basis.quantize(gqcpy.OverlapOperator())
    
    rsq_hamiltonian = gqcpy.RSQHamiltonian_cd.Molecular(basis, molecule)
    objective = gqcpy.DiagonalRHFFockMatrixObjective_cd(rsq_hamiltonian, 1.0e-5)  
    
    environment = gqcpy.RHFSCFEnvironment_cd.WithComplexlyTransformedCoreGuess(N, rsq_hamiltonian, S)
    solver = gqcpy.RHFSCFSolver_cd.Plain(threshold=1.0e-01, maximum_number_of_iterations=10000)

    qc_structure = gqcpy.RHF_cd.optimize(objective, solver, environment)
    nuc_rep = gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()
    print(qc_structure.groundStateEnergy().real + nuc_rep)
    
    return qc_structure

In [26]:
complex_RHF_model = complex_RHF_calculation(H4).parameters()

RuntimeError: IterativeAlgorithm<Environment>::perform(Environment&): The algorithm didn't find a solution within the maximum number of iterations.

In [ ]:
restricted_basis = gqcpy.RSpinOrbitalBasis_cd(H4, 'sto-3g')
restricted_hamiltonian = gqcpy.RSQHamiltonian_cd.Molecular(restricted_basis, H4)
rsq_hamiltonian_mo = restricted_hamiltonian.transformed(complex_RHF_model.expansion())
restricted_stability_matrices = complex_RHF_model.calculateStabilityMatrices(rsq_hamiltonian_mo)

In [ ]:
restricted_stability_matrices.isInternallyStable()

## The UHF stability conditions

We will run an UHF calculation on $H_3$.

In [27]:
def UHF_calculation(molecule, basis_set='STO-3G'):
    N_a = molecule.numberOfElectronPairs()
    N_b = molecule.numberOfElectrons() - N_a
    basis = gqcpy.USpinOrbitalBasis_d(molecule, basis_set)
    S = basis.quantize(gqcpy.OverlapOperator())
    sq_hamiltonian = gqcpy.USQHamiltonian_d.Molecular(basis, molecule) 

    environment = gqcpy.UHFSCFEnvironment_d.WithCoreGuess(N_a, N_b, sq_hamiltonian, S) 
    solver = gqcpy.UHFSCFSolver_d.Plain(1.0e-06, 3000)

    qc_structure = gqcpy.UHF_d.optimize(solver, environment)
    
    return qc_structure

In [28]:
H3 = gqcpy.Molecule.HRingFromDistance(3, 1.889, 0)

In [29]:
UHF_model = UHF_calculation(H3).parameters()

These parameters contain everything there is to know about the UHF wave function model. We can now ask this model to calculate the stability matrices. For that we need a Hamiltonian in the UHF MO basis.

In [30]:
unrestricted_basis = gqcpy.USpinOrbitalBasis_d(H3, 'STO-3G')
usq_hamiltonian_mo = gqcpy.USQHamiltonian_d.Molecular(unrestricted_basis, H3).transformed(UHF_model.expansion())
unrestricted_stability_matrices = UHF_model.calculateStabilityMatrices(usq_hamiltonian_mo)

Now we can check the stabilities. 

In [31]:
unrestricted_stability_matrices.isInternallyStable()

True

In [32]:
unrestricted_stability_matrices.isExternallyStable()

False

In [33]:
unrestricted_stability_matrices.isSpinUnconservedStable()

False

In [34]:
unrestricted_stability_matrices.isComplexConjugateStable()

True

In [35]:
unrestricted_stability_matrices.printStabilityDescription()

The real valued UHF wavefunction is internally stable.
The real valued UHF wavefunction is stable within the real/complex UHF space.
The real valued UHF wavefunction contains an unrestricted->generalized instability.


## The GHF stability conditions

We will run an GHF calculation on $H_3$.

In [36]:
def real_GHF_calculation(molecule, basis_set='STO-3G'):
    N = H3.numberOfElectrons()
    basis = gqcpy.GSpinorBasis_d(H3, basis_set)
    S = basis.quantize(gqcpy.OverlapOperator())
    gsq_hamiltonian = gqcpy.GSQHamiltonian_d.Molecular(basis, molecule) 

    environment = gqcpy.GHFSCFEnvironment_d.WithCoreGuess(N, gsq_hamiltonian, S) 
    solver = gqcpy.GHFSCFSolver_d.Plain(1.0e-08, 4000)
    qc_structure = gqcpy.GHF_d.optimize(solver, environment)
    nuc_rep = gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()
    print(qc_structure.groundStateEnergy() + nuc_rep)
    
    return qc_structure

In [37]:
GHF_model = real_GHF_calculation(H3).parameters()

-1.3358471594598829


These parameters contain everything there is to know about the GHF wave function model. We can now ask this model to calculate the stability matrices. For that we need a Hamiltonian in the GHF MO basis.

In [38]:
generalized_basis = gqcpy.GSpinorBasis_d(H3, 'STO-3G')
gsq_hamiltonian_mo = gqcpy.GSQHamiltonian_d.Molecular(generalized_basis, H3).transformed(GHF_model.expansion())
generalized_stability_matrices = GHF_model.calculateStabilityMatrices(gsq_hamiltonian_mo)

In [39]:
generalized_stability_matrices.isInternallyStable()

False

In [28]:
generalized_stability_matrices.isExternallyStable()

False

In [29]:
generalized_stability_matrices.printStabilityDescription()

The real valued GHF wavefunction contains an internal instability.
The real valued GHF wavefunction contains a real->complex external instability.


Since we noticed that there's a real->complex external instability, let's try to find a complex GHF solution that is lower in energy.

In [30]:
def complex_GHF_calculation(molecule, basis_set='STO-3G'):
    N = H3.numberOfElectrons()
    basis = gqcpy.GSpinorBasis_cd(H3, basis_set)
    S = basis.quantize(gqcpy.OverlapOperator())
    gsq_hamiltonian = gqcpy.GSQHamiltonian_cd.Molecular(basis, molecule) 

    environment = gqcpy.GHFSCFEnvironment_cd.WithComplexlyTransformedCoreGuess(N, gsq_hamiltonian, S) 

    solver = gqcpy.GHFSCFSolver_cd.Plain(threshold=1.0e-08, maximum_number_of_iterations=4000)
    qc_structure = gqcpy.GHF_cd.optimize(solver, environment)
    nuc_rep = gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()
    print(qc_structure.groundStateEnergy().real + nuc_rep)

    return qc_structure

In [31]:
complex_GHF_model = complex_GHF_calculation(H3).parameters()

-1.3403026284286494


In [32]:
generalized_basis = gqcpy.GSpinorBasis_cd(H3, "STO-3G")
gsq_hamiltonian_mo = gqcpy.GSQHamiltonian_cd.Molecular(generalized_basis, H3).transformed(complex_GHF_model.expansion())
generalized_stability_matrices = complex_GHF_model.calculateStabilityMatrices(gsq_hamiltonian_mo)

In [33]:
generalized_stability_matrices.printStabilityDescription()

The complex valued GHF wavefunction is internally stable.
